<a href="https://colab.research.google.com/github/Rocking-Priya/704-fall-projects-2025/blob/main/project_(4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DX 704 Week 7 Project
This week's project will investigate issues in a quadcopter controller based using a linear quadratic regulator.
You will start with an existing model of the system and logs from a quadcopter based on it, investigate discrepancies, and ultimately train a new model of the system dynamics.

The full project description and a template notebook are available on GitHub: [Project 7 Materials](https://github.com/bu-cds-dx704/dx704-project-07).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Introduction

You've just joined a drone startup.
On your first day, you join your new team to watch a test flight for a new quadcopter prototype.
Watching the prototype fly, the team comments that it is not as smooth as usual and suspects that something is off in the controller.
They provide you a copy of the dynamics model and log data from the prototype to investigate.

The quadcopter control model is a slightly more sophisticated version of the 1D quadcopter problem previously considered.

The state vector $\mathbf{x}_t$ now includes an acceleration component, and the action vector now has a servo control for the throttle instead of a raw force component.
\begin{array}{rcl}
\mathbf{x}_t & = & \begin{bmatrix} x_t \\ v_t \\ a_t \end{bmatrix} \\
\mathbf{u_t} & = & \begin{bmatrix} u_t \end{bmatrix}
\end{array}

## Part 1: Reconstruct the Control Matrix

You are provided the dynamics model in the files `model-A.tsv`, `model-B.tsv`, `cost-Q.tsv` and `cost-R.tsv`.
Recompute the control matrix $\mathbf{K}$ to be used in the infinite horizon linear quadratic regulator problem.

In [2]:
%pip install -q control

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.3/578.3 kB 5.7 MB/s eta 0:00:00


In [24]:
import numpy as np
import pandas as pd
import control
from scipy.linalg import solve_discrete_are

In [19]:
A_df = pd.read_csv('model-A.tsv', sep='\t', header=None)
B_df = pd.read_csv('model-B.tsv', sep='\t', header=None)  # YOUR CHANGES HERE
Q_df = pd.read_csv('cost-Q.tsv', sep='\t', header=None)
R_df = pd.read_csv('cost-R.tsv', sep='\t', header=None)


In [21]:
print(A_df)
print(B_df)
print(Q_df)
print(R_df)

       0  1  2  3
0  index  x  v  a
1      x  1  1  0
2      v  0  1  1
3      a  0  0  1
       0  1
0  index  u
1      x  0
2      v  0
3      a  1
       0  1  2  3
0  index  x  v  a
1      x  5  0  0
2      v  0  1  0
3      a  0  0  2
       0  1
0  index  u
1      u  5


In [22]:
#Drop the first row and first column which are the textual index/header in your files
#    and convert to numeric numpy arrays.
A = A_df.iloc[1:, 1:].astype(float).values    # shape (n, n)
B = B_df.iloc[1:, 1:].astype(float).values    # shape (n, m)
Q = Q_df.iloc[1:, 1:].astype(float).values    # shape (n, n)
R = R_df.iloc[1:, 1:].astype(float).values    # shape (m, m)

In [23]:
print("Shapes: A", A.shape, "B", B.shape, "Q", Q.shape, "R", R.shape)

Shapes: A (3, 3) B (3, 1) Q (3, 3) R (1, 1)


In [25]:
# 3) Solve the discrete Algebraic Riccati Equation (DARE)
P = solve_discrete_are(A, B, Q, R)

In [26]:
# 4) Compute LQR gain K (discrete-time formula)
#    K = (B' P B + R)^{-1} (B' P A)
BT_P_B = B.T @ P @ B
inv_term = np.linalg.inv(BT_P_B + R)   # R should be positive definite for stability/numerical reasons
K = inv_term @ (B.T @ P @ A)

In [27]:
print("\nComputed K:")
print(K)


Computed K:
[[0.33445985 1.30445413 1.85813088]]


Save $\mathbf{K}$ in a file "control-K-intended.tsv" with columns x, v and a.

In [28]:
# YOUR CHANGES HERE
K = pd.DataFrame(K, columns=['x', 'v', 'a'])
K.to_csv('control-K-intended.tsv', sep='\t', index=False)


In [29]:
from google.colab import files


In [30]:
files.download('control-K-intended.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Submit "control-K-intended.tsv" in Gradescope.

## Part 2: Recompute the Actions for the Logged States

You get access to the log data for the prototype as the file "qc-log.tsv".
It conveniently saves all the state and actions made.
Recompute the actions based on your $\mathbf{K}$ matrix computed in part 1.

In [31]:
# YOUR CHANGES HERE
qc_log = pd.read_csv('qc-log.tsv', sep='\t')


In [32]:
print(qc_log)

    index             x             v             a             u
0       0 -5.000000e+00  0.000000e+00  0.000000e+00  1.702188e+00
1       1 -5.000000e+00 -1.702188e-02  1.531969e+00 -1.263200e+00
2       2 -5.018724e+00  1.452683e+00  5.482855e-01 -1.249321e+00
3       3 -3.420773e+00  1.840779e+00 -5.212749e-01 -2.121274e-01
4       4 -1.395916e+00  1.163611e+00 -7.643170e-01  4.528954e-01
..    ...           ...           ...           ...           ...
95     95 -1.855920e-18  1.097460e-18 -1.592936e-19 -4.843645e-19
96     96 -6.487133e-19  8.412290e-19 -6.111510e-19  3.189634e-19
97     97  2.766386e-19  1.733230e-19 -3.851990e-19  4.316642e-19
98     98  4.672940e-19 -2.142650e-19 -3.522116e-20  1.881712e-19
99     99  2.316025e-19 -2.281803e-19  1.306108e-19 -3.689071e-20

[100 rows x 5 columns]


In [37]:
# compute_and_check_lqr.py

from pathlib import Path
from scipy.linalg import solve_discrete_are

def load_numeric_block(path):
    """
    Load a TSV that may contain textual header/index row/col.
    Returns the largest numeric submatrix found by:
      - reading everything as strings,
      - converting to numeric with errors='coerce' (non-numeric -> NaN),
      - dropping rows and columns that are all NaN,
      - returning the remaining numeric values as np.array (float).
    """
    df_raw = pd.read_csv(path, sep='\t', header=None, dtype=str)
    # convert all cells to numeric where possible
    df_num = df_raw.applymap(lambda v: pd.to_numeric(v, errors='coerce'))
    # drop rows and columns that are entirely NaN
    df_num = df_num.dropna(axis=0, how='all').dropna(axis=1, how='all')
    if df_num.size == 0:
        raise ValueError(f"No numeric data found in {path}")
    # Re-check: if first row or col are non-numeric labels that survived, we will still have numeric block.
    return df_num.astype(float).values

# --- Part 1: compute K and save control-K-intended.tsv ---
A_path = Path("model-A.tsv")
B_path = Path("model-B.tsv")
Q_path = Path("cost-Q.tsv")
R_path = Path("cost-R.tsv")

if not (A_path.exists() and B_path.exists() and Q_path.exists() and R_path.exists()):
    raise FileNotFoundError("One or more model files missing. Need: model-A.tsv, model-B.tsv, cost-Q.tsv, cost-R.tsv")

A = load_numeric_block(A_path)
B = load_numeric_block(B_path)
Q = load_numeric_block(Q_path)
R = load_numeric_block(R_path)

# Validate shapes
n = A.shape[0]
if A.shape != (n, n):
    raise ValueError(f"A must be square; got {A.shape}")
if B.shape[0] != n:
    raise ValueError(f"B must have same number of rows as A; got A{A.shape} B{B.shape}")
if Q.shape != (n, n):
    raise ValueError(f"Q must be {n}x{n}; got {Q.shape}")
if R.shape[0] != R.shape[1]:
    raise ValueError(f"R must be square; got {R.shape}")

# Solve DARE and compute discrete-time LQR gain K
P = solve_discrete_are(A, B, Q, R)            # P
K = np.linalg.inv(B.T @ P @ B + R) @ (B.T @ P @ A)  # shape (m, n)

# Save K with columns named x, v, a (assumes state order is x, v, a)
# If state dimension isn't 3, adjust column names accordingly.
n_state = K.shape[1]
colnames = ['x', 'v', 'a'][:n_state] if n_state <= 3 else [f"s{i}" for i in range(n_state)]
K_df = pd.DataFrame(K, columns=colnames)
K_df.to_csv('control-K-intended.tsv', sep='\t', index=False, float_format='%.12g')
print("Saved control-K-intended.tsv:")
print(K_df.to_string(index=False))

# --- Part 2: recompute actions for logged states ---
qc_log_path = Path("qc-log.tsv")
if not qc_log_path.exists():
    raise FileNotFoundError("qc-log.tsv not found. Please place the log file in the same folder.")

qc_log = pd.read_csv(qc_log_path, sep='\t')  # assume standard csv with header row
print("\nqc-log columns:", qc_log.columns.tolist())

# Find state columns - try common names 'x','v','a'. If not there, guess numeric columns after 'index'.
expected_state_cols = ['x', 'v', 'a']
if set(expected_state_cols).issubset(qc_log.columns):
    state_cols = expected_state_cols
else:
    # fallback: take numeric columns excluding 'u' and 'index' if present
    numeric_cols = [c for c in qc_log.columns if np.issubdtype(qc_log[c].dtype, np.number)]
    # try to pick first three numeric columns as states
    if len(numeric_cols) >= 3:
        # prefer column names that look like x,v,a
        state_cols = numeric_cols[:3]
    else:
        raise ValueError("Could not identify state columns (x, v, a) in qc-log.tsv. Columns: " + ", ".join(qc_log.columns))

states = qc_log[state_cols].astype(float).values  # shape (N, n_state)

# Ensure K matches state dimension
if K.shape[1] != states.shape[1]:
    raise ValueError(f"Dimension mismatch: K has {K.shape[1]} cols but log states have {states.shape[1]} entries.")

# Compute u_check = -K @ x_t for each row
u_check_vals = - (K @ states.T).T   # shape (N, m)
# For single-action controllers, flatten
if u_check_vals.shape[1] == 1:
    u_check_flat = u_check_vals.ravel()
else:
    # If multiple actions, we save the first column (or you can save all columns).
    u_check_flat = u_check_vals[:, 0]
    print("Warning: multiple action outputs from K; saving first output as u_check.")


# Optional comparison if logged 'u' exists
if 'u' in qc_log.columns:
    logged_u = qc_log['u'].astype(float).values
    if len(logged_u) == len(u_check_flat):
        diffs = logged_u - u_check_flat
        rmse = np.sqrt(np.mean(diffs**2))
        mean_abs = np.mean(np.abs(diffs))
        print(f"\nComparison with logged 'u': RMSE = {rmse:.6g}, mean abs diff = {mean_abs:.6g}")
    else:
        print("Logged 'u' present but length mismatch; skipped comparison.")

# Quick closed-loop eigenvalues check
Acl = A - B @ K
eigvals = np.linalg.eigvals(Acl)
print("\nClosed-loop eigenvalues (A - B K):", eigvals)
print("Magnitudes:", np.abs(eigvals))
if np.all(np.abs(eigvals) < 1.0):
    print("Closed-loop (discrete) is stable (all |eig| < 1).")
else:
    print("Warning: closed-loop may be unstable (some |eig| >= 1).")


Saved control-K-intended.tsv:
      x        v        a
0.33446 1.304454 1.858131

qc-log columns: ['index', 'x', 'v', 'a', 'u']

Comparison with logged 'u': RMSE = 0.0168146, mean abs diff = 0.0038978

Closed-loop eigenvalues (A - B K): [0.36935751+0.j         0.3862558 +0.39200236j 0.3862558 -0.39200236j]
Magnitudes: [0.36935751 0.55032663 0.55032663]
Closed-loop (discrete) is stable (all |eig| < 1).


/tmp/ipython-input-3740110411.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_num = df_raw.applymap(lambda v: pd.to_numeric(v, errors='coerce'))
/tmp/ipython-input-3740110411.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_num = df_raw.applymap(lambda v: pd.to_numeric(v, errors='coerce'))
/tmp/ipython-input-3740110411.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_num = df_raw.applymap(lambda v: pd.to_numeric(v, errors='coerce'))
/tmp/ipython-input-3740110411.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_num = df_raw.applymap(lambda v: pd.to_numeric(v, errors='coerce'))


Save your computed actions as "qc-check.tsv" with columns "index" and "u_check".

In [38]:
# YOUR CHANGES HERE
# Build result dataframe
out_df = pd.DataFrame({
    'index': qc_log['index'] if 'index' in qc_log.columns else np.arange(len(qc_log)),
    'u_check': u_check_flat
})
out_df.to_csv('qc-check.tsv', sep='\t', index=False, float_format='%.12g')
print("\nSaved qc-check.tsv (index, u_check)")



Saved qc-check.tsv (index, u_check)


In [39]:
files.download('qc-check.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Submit "qc-check.tsv" in Gradescope.

## Part 3: Reverse Engineer the Actual Control Matrix

Now that you have found a systematic difference between your computed actions and the logged actions, estimate $
$, the control matrix that was actually used to choose actions in the prototype.

Hint: With a linear quadratic regulator, the optimal actions are always linear combinations of the state that are calculated using the control matrix.
You can use linear regression to reverse-engineer the coefficients in the control matrix.

In [43]:
# YOUR CHANGES HERE

# --- helper: load K_intended if present (optional) ---
def load_K_intended(path="control-K-intended.tsv"):
    p = Path(path)
    if not p.exists():
        return None
    df = pd.read_csv(p, sep='\t')
    return df.values  # shape (m, n)

# --- 1) load qc-log ---
qc_path = Path("qc-log.tsv")
if not qc_path.exists():
    raise FileNotFoundError("qc-log.tsv not found. Place the log file here.")

qc = pd.read_csv(qc_path, sep='\t')
# Expect 'index', 'x', 'v', 'a', 'u' columns (but attempt to be flexible)
if not set(['x','v','a']).issubset(qc.columns):
    raise ValueError("qc-log.tsv must contain columns 'x','v','a'.")

# Build design matrix X and target y
X = qc[['x','v','a']].astype(float).values   # shape (N, 3)
y = qc['u'].astype(float).values.reshape(-1, 1)  # shape (N, 1)

# --- 2) solve linear least squares for beta in y ≈ X beta^T (here y = X @ beta + eps)
# We want beta shape (3,1) such that y ≈ X @ beta
beta, residuals, rank, s = np.linalg.lstsq(X, y, rcond=None)  # beta shape (3,1)

# beta is coefficients so y ≈ X @ beta. LQR law is u = -K x -> beta = -K^T
# So K_actual (shape (1,3)) = - beta.T
K_actual = - beta.T  # shape (1, 3)

print("Estimated linear coefficients beta (for u ≈ X @ beta):")
print(beta.ravel())
print("\nDerived K_actual = -beta^T:")
print(K_actual)

# --- 3) Diagnostics: fit quality ---
y_hat = X @ beta        # shape (N,1)
residuals = (y - y_hat).ravel()
rmse = np.sqrt(np.mean(residuals**2))
mae = np.mean(np.abs(residuals))
print(f"\nFit diagnostics: RMSE = {rmse:.6g}, MAE = {mae:.6g}, rank = {rank}")



Estimated linear coefficients beta (for u ≈ X @ beta):
[-0.34043755 -1.30012023 -1.95011696]

Derived K_actual = -beta^T:
[[0.34043755 1.30012023 1.95011696]]

Fit diagnostics: RMSE = 1.12043e-16, MAE = 3.08966e-17, rank = 3


Save $\mathbf{K}_{\mathrm{actual}}$ in "control-K-actual.tsv" with the same format as "control-K-intended.tsv".

In [48]:
K_df.to_csv('control-K-actual.tsv', sep='\t', index=False, float_format='%.12g')

In [46]:
# YOUR CHANGES HERE

colnames = ['x','v','a']
K_actual_df = pd.DataFrame(K_actual, columns=colnames)  # 1 x 3
K_actual_df.to_csv('actual-K.tsv', sep='\t', index=False, float_format='%.12g')
print("\nSaved actual-K.tsv:")
print(K_actual_df.to_string(index=False))



Saved actual-K.tsv:
       x       v        a
0.340438 1.30012 1.950117


In [47]:
files.download('actual-K.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Submit "actual-K.tsv" in Gradescope.

## Part 4: Recompute the System Dynamics from the Log Data

On further investigation, it turns out that the control matrix $\mathbf{K}$ in the prototype was modified to compensate for state prediction errors.
You would like to recompute the $\mathbf{A}$ and $\mathbf{B}$ matrices using the log data since they are used to predict the next states.
However, since the action vector $\mathbf{u}_t$ is linearly dependent on the state via $\mathbf{u}_t=-\mathbf{K} \mathbf{x}_t$, you need a new data set so you can separate the effects of the $\mathbf{A}$ and $\mathbf{B}$ matrices.
Your co-workers kindly provide a new file "qc-train.tsv" where noise was added to each action.
Estimate the true $\mathbf{A}$ and $\mathbf{B}$ matrices based on this file.

In [49]:
# YOUR CHANGES HERE

qc_train = pd.read_csv('qc-train.tsv', sep='\t')

In [50]:
print(qc_train)

    index         x         v         a         u
0       0 -5.000000  0.000000  0.000000  1.729856
1       1 -5.000000 -0.017299  1.556871 -1.311911
2       2 -5.019028  1.476577  0.531837 -1.198850
3       3 -3.394793  1.846154 -0.493944 -0.297565
4       4 -1.364024  1.195267 -0.811147  0.472619
..    ...       ...       ...       ...       ...
95     95  0.089539 -0.162778  0.122030 -0.008091
96     96 -0.089517 -0.030491  0.126952 -0.174120
97     97 -0.123056  0.094904 -0.017061  0.038104
98     98 -0.018662  0.068824  0.015527 -0.062828
99     99  0.057045  0.077321 -0.039466 -0.015137

[100 rows x 5 columns]


In [51]:
# --- 1) load qc-train.tsv ---
p = Path("qc-train.tsv")
if not p.exists():
    raise FileNotFoundError("qc-train.tsv not found in working directory.")

df = pd.read_csv(p, sep='\t')
print("Columns found:", df.columns.tolist())
# Expect 'index','x','v','a','u' (but be flexible)
required = ['x','v','a','u']
if not set(['x','v','a']).issubset(df.columns):
    # If headers are different, you may need to inspect df.head()
    raise ValueError("qc-train.tsv must contain numeric columns for x, v, a (and u).")

# --- 2) construct X_t, U_t, X_{t+1} ---
# Use rows 0..N-2 as t, rows 1..N-1 as t+1
states = df[['x','v','a']].astype(float).values   # shape (N, 3)
actions = df['u'].astype(float).values.reshape(-1, 1)  # shape (N, 1)

# drop last row for t and drop first row for t+1
X_t = states[:-1, :]        # (N-1, 3)
U_t = actions[:-1, :]       # (N-1, 1)
X_tp1 = states[1:, :]       # (N-1, 3)

N_minus_1 = X_t.shape[0]
print(f"Using {N_minus_1} data rows for regression (N-1).")

# --- 3) Build design matrix and solve via least-squares ---
# Design matrix: [x, v, a, u] shape (N-1, 4)
D = np.hstack([X_t, U_t])   # shape (N-1, 4)

# Solve D * M = X_tp1 for M (4 x 3) using least squares.
# Each column j in M contains the coefficients mapping D to X_{t+1}[:, j].
M, residuals, rank, s = np.linalg.lstsq(D, X_tp1, rcond=None)

# Extract A and B
# As derived: for each next-state component j, M[:, j] = [A_j1, A_j2, A_j3, B_j]^T
# So A (3x3) has rows A_j = M[0:3, j]^T  (i.e., A = M[0:3, :].T)
A_est = M[0:3, :].T   # shape (3,3)
B_est = M[3, :].reshape((3, 1))  # shape (3,1)

print("\nEstimated A matrix (rows = next-state components):\n", A_est)
print("\nEstimated B vector (column):\n", B_est)

# --- 4) Diagnostics: compute residuals and RMSE ---
pred = D @ M           # shape (N-1, 3)
errors = X_tp1 - pred  # residuals (N-1, 3)
rmse_per_dim = np.sqrt(np.mean(errors**2, axis=0))
rmse_total = np.sqrt(np.mean(errors.ravel()**2))
print("\nRMSE per next-state component:", rmse_per_dim)
print("Overall RMSE:", rmse_total)



Columns found: ['index', 'x', 'v', 'a', 'u']
Using 99 data rows for regression (N-1).

Estimated A matrix (rows = next-state components):
 [[ 1.00000000e+00  1.10000000e+00  2.88657986e-15]
 [-1.28195498e-16  9.00000000e-01  9.50000000e-01]
 [-9.31378031e-17  5.55111512e-16  1.10000000e+00]]

Estimated B vector (column):
 [[-1.11022302e-16]
 [-1.00000000e-02]
 [ 9.00000000e-01]]

RMSE per next-state component: [9.48993751e-16 3.37486040e-16 2.55714247e-16]
Overall RMSE: 5.999654830176132e-16


Save $\mathbf{A}$ and $\mathbf{B}$ to "model-A-new.tsv" and "model-B-new.tsv" respectively.

In [52]:
# YOUR CHANGES HERE

# --- 5) Save results to TSV files in same format as earlier (no index column) ---
A_df = pd.DataFrame(A_est, columns=['x','v','a'])  # rows correspond to next-state eqn 1..3
A_df.to_csv('model-A-new.tsv', sep='\t', index=False, float_format='%.12g')

B_df = pd.DataFrame(B_est, index=['x','v','a'], columns=['u'])  # align rows with states
B_df.to_csv('model-B-new.tsv', sep='\t', index=True, float_format='%.12g')

print("\nSaved 'model-A-new.tsv' and 'model-B-new.tsv'.")


Saved 'model-A-new.tsv' and 'model-B-new.tsv'.


Submit "model-A-new.tsv" and "model-B-new.tsv" in Gradescope.

In [53]:
files.download('model-A-new.tsv')
files.download('model-B-new.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Part 5: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 6: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.